In [96]:
import nltk
import re

In [10]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#import plotly.express as px

from pandas_profiling import ProfileReport
from sklearn import metrics
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBClassifier

In [11]:
KAGGLE_EVAL_METRIC = 'logloss' # string name for loss function in xgboost

In [195]:
TRAIN_PATH = "data/train.csv"
TEST_PATH = "data/test.csv"

# Load Data

In [201]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

## TESTS

In [202]:
# NLP

In [222]:
## Helper functions
def get_nickanme(x):
    if "(" in x:
        nickname = re.findall('\(.*?\)',x)[0]
    elif '"' in x:
        nickname = re.findall('\".*?\"',x)[0]
    else:
        nickname = "none"
    return nickname

def get_lastname(name, first_name, title, nickname):
    last_name = name.replace(first_name,'')
    last_name = last_name.replace(title,'')
    last_name = last_name.replace(nickname,'')
    last_name = last_name.replace(',','')
    return last_name

def replace_last(source_string, replace_what):
    head, _sep, tail = source_string.rpartition(replace_what)
    return head + "" + tail


In [223]:
train.Name = train.Name.str.lower()

## first name, title, nickname
train['First Name'] = train['Name'].apply(lambda x: x.split(",")[0])
train['Title'] = train['Name'].apply(lambda x: x.split(",")[1].split()[0])
train['Nickname'] = train['Name'].apply(lambda x: get_nickanme(x))
train['Last Name'] = train.apply(lambda x: get_lastname(x['Name'], x['First Name'],x['Title'],x['Nickname']), axis=1)
train['junior'] = train['Last Name'].apply(lambda x: x.endswith(("jr","ii")))
train['Last Name'] = train['Name'].apply(lambda x: replace_last(x, "jr"))
train['Last Name'] = train['Name'].apply(lambda x: replace_last(x, "ii"))

In [225]:
train[train['Name'].str.contains("ii")]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,First Name,Title,Nickname,Last Name,temp,temp2,junior
50,51,0,3,"panula, master. juha niilo",male,7.0,4,1,3101295,39.6875,NaN,S,panula,master.,none,"panula, master. juha nlo","[juha, niilo]",2,False
113,114,0,3,"jussila, miss. katriina",female,20.0,1,0,4136,9.8250,NaN,S,jussila,miss.,none,"jussila, miss. katrna",[katriina],1,False
216,217,1,3,"honkanen, miss. eliina",female,27.0,0,0,STON/O2. 3101283,7.9250,NaN,S,honkanen,miss.,none,"honkanen, miss. elna",[eliina],1,False
417,418,1,2,"silven, miss. lyyli karoliina",female,18.0,0,2,250652,13.0000,NaN,S,silven,miss.,none,"silven, miss. lyyli karolna","[lyyli, karoliina]",2,False
579,580,1,3,"jussila, mr. eiriik",male,32.0,0,0,STON/O 2. 3101286,7.9250,NaN,S,jussila,mr.,none,"jussila, mr. eirk",[eiriik],1,False
702,703,0,3,"barbara, miss. saiide",female,18.0,0,1,2691,14.4542,NaN,C,barbara,miss.,none,"barbara, miss. sade",[saiide],1,False
761,762,0,3,"nirva, mr. iisakki antino aijo",male,41.0,0,0,SOTON/O2 3101272,7.1250,NaN,S,nirva,mr.,none,"nirva, mr. sakki antino aijo","[iisakki, antino, aijo]",3,False
802,803,1,1,"carter, master. william thornton ii",male,11.0,1,2,113760,120.0000,B96 B98,S,carter,master.,none,"carter, master. william thornton","[william, thornton, ii]",3,True
867,868,0,1,"roebling, mr. washington augustus ii",male,31.0,0,0,PC 17590,50.4958,A24,S,roebling,mr.,none,"roebling, mr. washington augustus","[washington, augustus, ii]",3,True
